In [ ]:
pip install pyldavis

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 187 kB/s 
     |████████████████████████████████| 11.3 MB 39.5 MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=ad00b029f844f3e3cf803905fb6d3bda8ed875b78b287dc152aa90d2d9a24a4f
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyldavis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re
from sklearn.model_selection import train_test_split
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from string import digits,punctuation
from sklearn.decomposition import LatentDirichletAllocation
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colaboratory Stuff/Chats.csv')

In [ ]:
pd.set_option('Max_colwidth',-1)
df.drop('Clean',axis=1,inplace=True)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,Unnamed: 0,Text
0,0,What advantage is there in booking directly with an airline rather than through an agent? I have almost always booked with the airline but now have an agent whose price is around $50 cheaper than airline and airline does not have price match.
1,1,"Hi, Could someone please confirm if CX 884 - HKG-LAX business class seats are truly lie flat seats? The aircraft code shows 773, travel agent says they are lie flat but Seatguru shows them as recliners. Help is appreciated. Cheers"
2,2,I will be transiting Dubai soon en route to Oz. Never flown anywhere via Dubai before so am a bit wary about alcohol......not drinking it but having it in hand luggage. If I buy a bottle after security screening at LHR will it be a problem at any screening at Dubai before my connecting flight? It is a special gift requested by a friend in Oz.....but don't want to give myself any bother at airports...they are stressful enough places. Any advice?
3,3,"Does anyone know where I'd find estimated prices from Atlanta. My vacation destination is up in the air at this point and I'm flexible so I'd like to find a listing of places that are cheap to fly. Mexico, carribean, central american are all good choices. Thanks in advance."
4,4,It's from BA and finds the cheapest BA flight on a route britishairways.com/travel/…en_gb


In [ ]:
rows_to_remove=[row for row in range(len(df)) if df.iloc[row].isnull()['Text']==True]

In [ ]:
df.drop(rows_to_remove,axis=0,inplace=True)

In [ ]:
X_train,X_test=train_test_split(df, test_size=0.4, random_state=0)

In [ ]:
X_train.set_index(pd.Index(range(X_train.shape[0])),inplace=True)
X_train.shape

(48664, 2)

In [ ]:
ignored_words=['would','like','to','know','please','let','me','want','will','airways','airway', 'need','trying','try',
               'airline','airlines','flight','ticket','new york','hong kong','british','york','kong','air','canada','franc','san',
               'diego','francisco','turkish','virgin','zealand','orlean','penn','star','allianc','thoma','cook','aer','pacif','cathay'
               'airlin','american','mexico','china','airlin','boston','washington','florida','portland','union','tri','sau','paulo','costa','rica'
               'kuala','lampur','doe','san','cname','pname','name','los','angeles','able','hello','question','could','las','vegas','europe','april']

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
wn=WordNetLemmatizer()
from nltk.corpus import wordnet
def get_wordnet_pos(word):
  tag=nltk.pos_tag([word])[0][1][0]
  dict={'J':wordnet.ADJ,
        "N":wordnet.NOUN,
        "V":wordnet.VERB,
        "R":wordnet.ADV}
  return dict.get(tag, wordnet.NOUN)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
stemmer=PorterStemmer()
remove_digits = str.maketrans('', '', digits)

def tokenizer(text):
    text=text.translate(remove_digits)
    text=text.lower()
    text=text.rstrip()
    text=text.lstrip()
    text=re.sub(r"[^\w\s]",' ',text)
    token=nltk.word_tokenize(text)
    words=[wn.lemmatize(word,get_wordnet_pos(word)) for word in token if not word in set(stopwords.words('english')) and not word in punctuation and not word in ['cname','name','pname']
           and not word in set(ignored_words) and len(word)>2]
    return (words)

In [ ]:
test=X_train.copy()
test.Text=test.Text.apply(tokenizer)

In [ ]:
test

,Unnamed: 0,Text
0,35617,"[rebooking, cannceled, book, frequent, flyer, mile, get, charge, additional, mile, understood, fee, change, make, first, time]"
1,71341,"[anyone, ryanair, put, flight, summer, cork, lanzarote, seem, flight, tenerife, fuerteventura, etc, sale, already, none, lanzarote, cork, thanks]"
2,12627,"[find, someone, train]"
3,29694,"[consider, cut, cable, expensive, anything, help]"
4,21324,"[best, way, deal, get, buffalo, fairbanks, return, buffalo, vancuver, suggestion, help]"
...,...,...
48659,21283,"[friend, cut, credit, card, always, get, problem, much, spending, want, airplane, trip, book, internet, save, money, always, ask, credit, card, fee, book, travel, agent, airport, think, even, accept, check]"
48660,45981,"[mile, account, never, use, mile, use]"
48661,42698,"[get, bus, asap, bus, travel]"
48662,43652,"[train, time, today, clemente]"


In [ ]:

# bigrams=nltk.collocations.BigramAssocMeasures()
# finder=nltk.collocations.BigramCollocationFinder.from_documents([text for text in test.Text])
# finder.apply_freq_filter(75)
# finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
# bigram_score=finder.score_ngrams(bigrams.pmi)  
# bigram_pmi=pd.DataFrame(bigram_score,columns=['bigram','pmi score'])
# bigram_pmi.sort_values(by='pmi score',ascending=False, inplace=True)



In [ ]:
# bigram_pmi=bigram_pmi[bigram_pmi['pmi score']>2]

In [ ]:
# bigram_pmi[:50]

In [ ]:
def trigram_finder(test):
  trigrams=nltk.collocations.TrigramAssocMeasures()
  finder2=nltk.collocations.TrigramCollocationFinder.from_documents([text for text in test.Text])
  finder2.apply_freq_filter(20)
  finder2.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
  trigram_score=finder2.score_ngrams(trigrams.pmi)
  trigram_pmi=pd.DataFrame(trigram_score,columns=['trigram','pmi score'])
  trigram_pmi.sort_values(by='pmi score',ascending=False, inplace=True)
  return trigram_pmi

In [ ]:
trigram_pmi=trigram_finder(test)

In [ ]:
# bigrams=[]
# for i in range(len(bigram_pmi)):
#   w1,w2=bigram_pmi.iloc[i,0]
#   bigrams.append(w1+'_'+w2)

In [ ]:
# bigrams;

In [ ]:
trigrams=[]
for i in range(len(trigram_pmi)):
  x1,x2,x3=trigram_pmi.iloc[i,0]
  trigrams.append(x1+'_'+x2+'_'+x3)

In [ ]:
trigrams

['patient_bank_authenticates',
 'associate_document_tree',
 'lake_shore_limited',
 'bank_authenticates_transaction',
 'status_status_code_status_code',
 'visit_jeju_island',
 'arrives_dallas_departs',
 'xml_file_appear',
 'salt_lake_city',
 'contact_lens_solution',
 'complimentary_hot_meal',
 'west_palm_beach',
 'bbc_business_stm',
 'file_appear_style',
 'transparent_plastic_bag',
 'document_tree_show',
 'status_code_status_code_message',
 'information_associate_document',
 'style_information_associate',
 'message_xml_file',
 'anyone_shed_light',
 'associate_frequent_flyer',
 'match_associate_frequent',
 'either_milan_rome',
 'prevent_request_process',
 'status_code_message_forward',
 'tell_incorrect_zipcode',
 'system_issue_prevent',
 'reduce_monthly_fee',
 'news_bbc_business',
 'issue_prevent_request',
 'appear_style_information',
 'dallas_departs_hour',
 'modem_isnt_work',
 'tree_show_status',
 'departs_hour_stopover',
 'channel_available_shortly',
 'northeast_regional_train',
 'log

In [ ]:
def embed_trigrams(text):
  flag=[]
  i=0
  while i<(len(text)-2):
    gram=[]
    gram=[text[j] for j in range(i,i+3)]
    joined="_".join(gram)
    if joined in set(trigrams) and i!=len(text)-3:
      flag.append(joined)
      i+=3
      
    elif joined in set(trigrams) and i==len(text)-3:
      flag.append(joined)
      break
    
    elif joined not in set(trigrams) and i==len(text)-3:
      flag.append(text[i])
      flag.append(text[i+1])
      flag.append(text[i+2])
      break
 
    else:
      flag.append(text[i])
      i+=1
    

  return flag

In [ ]:
# def embed_trigrams2(text):
#   text_copy=text.copy()
#   all_trigrams=list(ngrams(text_copy,3))
  
#   for t in all_trigrams:
#     temp='_'.join(t)
#     if temp in set(trigrams):
#       ind=text_copy.index(t[0])
#       text_copy[ind:ind+3]=[temp]
#     # print(t)

#   return text_copy,text
    






In [ ]:
test.Text=test.Text.apply(embed_trigrams)

In [ ]:
test.head(50)

,Unnamed: 0,Text
0,35617,"[rebooking, cannceled, book, frequent_flyer_mile, get, charge, additional, mile, understood, fee, change, make, first, time]"
1,71341,"[anyone, ryanair, put, flight, summer, cork, lanzarote, seem, flight, tenerife, fuerteventura, etc, sale, already, none, lanzarote, cork, thanks]"
2,12627,"[find, someone, train]"
3,29694,"[consider, cut, cable, expensive, anything, help]"
4,21324,"[best, way, deal, get, buffalo, fairbanks, return, buffalo, vancuver, suggestion, help]"
5,26631,"[sent, refund, request, havent, herd, information, regard]"
6,59950,"[want, wifi, get, new, setup]"
7,74113,"[get_round_trip, ride, one, boarding, pas, two]"
8,38547,"[get, email, horrible, site]"
9,13219,"[get_well_price, husband, narfe, aarp, senior, auto, train, nov, roomette]"


In [ ]:
test.Text=test.Text.apply(lambda x: ' '.join(x))

In [ ]:
vectorizer=CountVectorizer(max_features=10000,ngram_range=(3,3))
vectors=vectorizer.fit_transform(test.Text)

In [ ]:
df=pd.DataFrame(vectors.toarray(),columns=vectorizer.get_feature_names())
df

,aaa say spouse,aaa thing card,abc ghj anybody,abreast anyone recently,absolute nightmare passenger,absolute rubbish travel,absolutely guess long,absolutely necessary look,absolutely record force,absolutely response anyone,absolutely wife return_round_trip,absorbed fee left,abta bond either,accept choice use,accept confirm employer,accept find information,accept mile time,accept term condition,accept zip code,access account online,access america refuse,access channel tablet,access email account,access email use,access mail keep_ask_sign,access message use,access mile line,access myway com,accidently sign rec,accord website inch,account accidently sign,account auto deduct,account check mile,account credit card,account error say,account fund related,account log payed,account mail physical,account neither see,account new account,...,yesterday flew cancun,yesterday istanbul stockholm,yesterday log account,yesterday morning call,yesterday print receipt,yesterday saw train,yesterday sometime around,yet account charge,yet also give,yet believe credit,yet get anyone,yet get email,yet possible funeral,yet thanks advance,yet website anyone,yiwu get mail,youtube com watch,yuo say reserve,yvr passenger party,ywg winnipeg delhi,ywg yyz somewhere,yyz somewhere del,yyz toronto india,zephyrtrain next september,zip code correct,zip code use,zip code yet,zipcode keep tell_incorrect_zipcode,zipcode put iit,zipcode sign tell_incorrect_zipcode,zipcode space bar,ziplock bag carry,zlo time baggage,zone grateful advice,zone land doha,zone paris thank,zoom day price,zurich jhb want,zurich leave zurich,zurich sydney bravofly
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48659,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48661,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48662,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
lda=LatentDirichletAllocation(n_components=5,max_iter=3,learning_method='online',n_jobs=-1,random_state=0)

In [ ]:
W1=lda.fit_transform(vectors)
H1=lda.components_

In [ ]:
top=10
vocab=np.array(vectorizer.get_feature_names())
top_words=lambda t: [vocab[i] for i in np.argsort(t)[:-top-1:-1]]
topic_words=[top_words(t) for t in H1]
topic=[",".join(t) for t in topic_words]

In [ ]:
topic

['round trip ticket,return trip outbound,xml_file_appear style_information_associate document_tree_show,told delta reservation,see redeem mile,use united mile,america refuse claim,access america refuse,email change password,account since longer',
 'long haul flight,one way ticket,recognize confirmation number,mail lot bad,way get phuket,use mile use,make_payment_arrangement pay full,full amount day,find train station,try reset password',
 'credit card always,trip book internet,always get problem,always ask credit,airport think even,get problem much,card always get,spending want airplane,cut credit card,airplane trip book',
 'go back sign,get_message_xml file_appear_style information_associate_document,service keep go,internet really slow,trans atlantic flight,ticket round trip,number phone number,say operate check,book month ago,cable box dvr',
 'log receive email,purchase roundtrip ticket,asap bus travel,get bus asap,bus asap bus,keep go back,phone day check,day check line,purchase ti

In [ ]:
# pip install pyldavis

In [ ]:
import pyLDAvis.sklearn

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.sklearn.prepare(lda, vectors, vectorizer)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.166480 -0.232922  1       1        20.953413
1     -0.347735 -0.017977  2       1        20.839000
0      0.065371 -0.070396  3       1        20.458516
4      0.114416  0.324536  4       1        19.461199
2      0.001468 -0.003241  5       1        18.287872, topic_info=                                                                  Term  ...  loglift
2128  credit card always                                                ...  30.0000
3778  go back sign                                                      ...  29.0000
4946  long haul flight                                                  ...  28.0000
7395  round trip ticket                                                 ...  27.0000
3756  get_message_xml file_appear_style information_associate_document  ...  26.0000
...                                                                ...  ...      ...
3700  get great deal                                                    ...  1.6686 
6254  option switch flight                                              ...  1.6686 
6291  overbooked see option                                             ...  1.6686 
8656  think might overbooked                                            ...  1.6686 
8444  switch flight reno                                                ...  1.6686 

[183 rows x 6 columns], token_table=      Topic  ...                                                            Term
term         ...                                                                
20    3      ...  access america refuse                                         
34    1      ...  account error say                                             
39    3      ...  account new account                                           
54    3      ...  account since longer                                          
97    1      ...  add mileageplan number                                        
...  ..      ...                   ...                                          
9669  2      ...  way get phuket                                                
9703  2      ...  website recognize confirmation                                
9728  3      ...  well business bureau                                          
9907  1      ...  wrong system take                                             
9922  3      ...  xml_file_appear style_information_associate document_tree_show

[153 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 5, 3])

In [ ]:
topic_labels={0:'Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password',
              1:'Long haul flight, one way ticket, miles/points usage, finding train station, payment, password reset',
              2: 'Credit card issues, internet problem, airport, airplane trip booking',
              3: 'Sign-in issues, xml message, service issues, slow internet, phone number, cable box, DVR',
              4: 'Bus travel, round trip ticket,urgent bus booking, online purchase of tickets,seat selection'}

In [ ]:
colnames=["Topic "+str(i) for i in range(lda.n_components)]
docs=["Doc"+str(i) for i in range(len(test.Text))]

In [ ]:
df_topic=pd.DataFrame(np.round(W1,2),columns=colnames,index=docs)
main_topic=np.argmax(df_topic.values,axis=1)
df_topic['Main Topic']=main_topic


In [ ]:
df_topic

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Main Topic
Doc0,0.10,0.10,0.10,0.10,0.60,4
Doc1,0.20,0.20,0.20,0.20,0.20,0
Doc2,0.10,0.10,0.10,0.60,0.10,3
Doc3,0.20,0.20,0.20,0.20,0.20,0
Doc4,0.20,0.20,0.20,0.20,0.20,0
...,...,...,...,...,...,...
Doc48659,0.01,0.01,0.97,0.01,0.01,2
Doc48660,0.10,0.60,0.10,0.10,0.10,1
Doc48661,0.05,0.05,0.05,0.05,0.80,4
Doc48662,0.20,0.20,0.20,0.20,0.20,0


In [ ]:
test.iloc[24,]

Unnamed: 0    39138                                                                                  
Text          believe ability connect internet curtail due unlawsful download may obtain use internet
Name: 24, dtype: object

In [ ]:
test['Topic']=df_topic['Main Topic'].values

In [ ]:
test.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
test

,Text,Topic,Keywords
0,rebooking cannceled book frequent_flyer_mile get charge additional mile understood fee change make first time,4,"log receive email,purchase roundtrip ticket,asap bus travel,get bus asap,bus asap bus,keep go back,phone day check,day check line,purchase ticket online,book select seat"
1,anyone ryanair put flight summer cork lanzarote seem flight tenerife fuerteventura etc sale already none lanzarote cork thanks,0,"round trip ticket,return trip outbound,xml_file_appear style_information_associate document_tree_show,told delta reservation,see redeem mile,use united mile,america refuse claim,access america refuse,email change password,account since longer"
2,find someone train,3,"go back sign,get_message_xml file_appear_style information_associate_document,service keep go,internet really slow,trans atlantic flight,ticket round trip,number phone number,say operate check,book month ago,cable box dvr"
3,consider cut cable expensive anything help,0,"round trip ticket,return trip outbound,xml_file_appear style_information_associate document_tree_show,told delta reservation,see redeem mile,use united mile,america refuse claim,access america refuse,email change password,account since longer"
4,best way deal get buffalo fairbanks return buffalo vancuver suggestion help,0,"round trip ticket,return trip outbound,xml_file_appear style_information_associate document_tree_show,told delta reservation,see redeem mile,use united mile,america refuse claim,access america refuse,email change password,account since longer"
...,...,...,...
48659,friend cut credit card always get problem much spending want airplane trip book internet save money always ask credit card fee book_travel_agent airport think even accept check,2,"credit card always,trip book internet,always get problem,always ask credit,airport think even,get problem much,card always get,spending want airplane,cut credit card,airplane trip book"
48660,mile account never use mile use,1,"long haul flight,one way ticket,recognize confirmation number,mail lot bad,way get phuket,use mile use,make_payment_arrangement pay full,full amount day,find train station,try reset password"
48661,get bus asap bus travel,4,"log receive email,purchase roundtrip ticket,asap bus travel,get bus asap,bus asap bus,keep go back,phone day check,day check line,purchase ticket online,book select seat"
48662,train time today clemente,0,"round trip ticket,return trip outbound,xml_file_appear style_information_associate document_tree_show,told delta reservation,see redeem mile,use united mile,america refuse claim,access america refuse,email change password,account since longer"


In [ ]:
test['Keywords']=np.NaN

In [ ]:
for i in range(len(test)):
  test.iloc[i,2]=topic_labels[test.iloc[i,1]]

In [ ]:
#test=test.sample(frac=1)
test.sample(frac=1)


,Text,Topic,Keywords
27288,still wait mile back last feb trip hawaii one part get upgrade lease mile back thank mileage plan,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
29508,read lot bad thing govolo anyone time confirmation pending yet receive confirmation yet paid make cancellation contact govolo govoyages take charge fee price thanks lot,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
27393,military fare price one way ticket austin antonio,1,"Long haul flight, one way ticket, miles/points usage, finding train station, payment, password reset"
26637,train nyc breakfast car,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
47190,dont mileage_plan_number,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
...,...,...,...
6662,ask question thomson dreamliners recent time maybe interested learn first plane arrive manchester around friday morning registration tuia regard dave,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
12993,ethihad kozhikode riyadh via abudhabi change much indian rupee change,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
45199,cancel drive hour home refund request,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
43693,last mileage_plan_account,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"


In [ ]:
X_train

,Unnamed: 0,Text
0,35617,"i am rebooking a cannceled flight that was booked with frequent flyer miles and I am getting charged and additional 12,500 miles. i understood there was no fee for changes made first time more than 2"
1,71341,"Hi, does anyone know if Ryanair are putting on flights for summer 2013 from Cork to Lanzarote??? They seem to have the flights to Tenerife, Fuerteventura etc. on sale already but none to Lanzarote from Cork. Thanks :)"
2,12627,How can I find out is someone is on the train or not?
3,29694,I'm considering cutting my cable off. It is so expensive. Is there anything you can do to help?
4,21324,What is the best way or deal to get from Buffalo NY to Fairbanks AL then return to Buffalo from Vancuver? Any suggestions would help...
...,...,...
48659,21283,I have a friend who has cut up ALL his Credit Cards because he was always getting into problems with to much spending. Now he wants to go on a Airplane trip and book on the Internet to save money. He is always asked for a Credit Card. There is a fee for booking with a Travel Agent or at the Airport and I do not think they even accept checks. Or do they?
48660,45981,I have no miles on my account.I never used my miles I want to use them
48661,42698,I need to get on a bus ASAP are there any other bus's i can travel on?
48662,43652,I need a train time for today from San Clemente to San Diego after 1:00 pm


In [ ]:
test

,Text,Topic,Keywords
0,rebooking cannceled book frequent_flyer_mile get charge additional mile understood fee change make first time,4,"log receive email,purchase roundtrip ticket,asap bus travel,get bus asap,bus asap bus,keep go back,phone day check,day check line,purchase ticket online,book select seat"
1,anyone ryanair put flight summer cork lanzarote seem flight tenerife fuerteventura etc sale already none lanzarote cork thanks,0,"round trip ticket,return trip outbound,xml_file_appear style_information_associate document_tree_show,told delta reservation,see redeem mile,use united mile,america refuse claim,access america refuse,email change password,account since longer"
2,find someone train,3,"go back sign,get_message_xml file_appear_style information_associate_document,service keep go,internet really slow,trans atlantic flight,ticket round trip,number phone number,say operate check,book month ago,cable box dvr"
3,consider cut cable expensive anything help,0,"round trip ticket,return trip outbound,xml_file_appear style_information_associate document_tree_show,told delta reservation,see redeem mile,use united mile,america refuse claim,access america refuse,email change password,account since longer"
4,best way deal get buffalo fairbanks return buffalo vancuver suggestion help,0,"round trip ticket,return trip outbound,xml_file_appear style_information_associate document_tree_show,told delta reservation,see redeem mile,use united mile,america refuse claim,access america refuse,email change password,account since longer"
...,...,...,...
48659,friend cut credit card always get problem much spending want airplane trip book internet save money always ask credit card fee book_travel_agent airport think even accept check,2,"credit card always,trip book internet,always get problem,always ask credit,airport think even,get problem much,card always get,spending want airplane,cut credit card,airplane trip book"
48660,mile account never use mile use,1,"long haul flight,one way ticket,recognize confirmation number,mail lot bad,way get phuket,use mile use,make_payment_arrangement pay full,full amount day,find train station,try reset password"
48661,get bus asap bus travel,4,"log receive email,purchase roundtrip ticket,asap bus travel,get bus asap,bus asap bus,keep go back,phone day check,day check line,purchase ticket online,book select seat"
48662,train time today clemente,0,"round trip ticket,return trip outbound,xml_file_appear style_information_associate document_tree_show,told delta reservation,see redeem mile,use united mile,america refuse claim,access america refuse,email change password,account since longer"


In [ ]:
final=test.copy()
final.Text=X_train.Text
final.head()

,Text,Topic,Keywords
0,"i am rebooking a cannceled flight that was booked with frequent flyer miles and I am getting charged and additional 12,500 miles. i understood there was no fee for changes made first time more than 2",4,"Bus travel, round trip ticket,urgent bus booking, online purchase of tickets,seat selection"
1,"Hi, does anyone know if Ryanair are putting on flights for summer 2013 from Cork to Lanzarote??? They seem to have the flights to Tenerife, Fuerteventura etc. on sale already but none to Lanzarote from Cork. Thanks :)",0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
2,How can I find out is someone is on the train or not?,3,"Sign-in issues, xml message, service issues, slow internet, phone number, cable box, DVR"
3,I'm considering cutting my cable off. It is so expensive. Is there anything you can do to help?,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
4,What is the best way or deal to get from Buffalo NY to Fairbanks AL then return to Buffalo from Vancuver? Any suggestions would help...,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"


In [ ]:
final.sample(n=10)

,Text,Topic,Keywords
5809,"why is your payment system not working, it keeps telling me to try later.",4,"Bus travel, round trip ticket,urgent bus booking, online purchase of tickets,seat selection"
43620,Just purchased tickets (round trip) MAF to IAH - IAH to MAF. It appears that two different credit cards were charged for these flights. please check and confirm that only one credit card was charged,3,"Sign-in issues, xml message, service issues, slow internet, phone number, cable box, DVR"
17885,Need person - what is a number I can try to call. In area with poor service. My phone ############ can get btext.,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
4763,"G'day everyone ,I'm sick of being charged top dollars for airtravel in this country (australia)and looking for cheap flights from melb australia to london uk but not purchasing from my departure point I recently saw flight london to melb ret for 485 pound including taxes and most are around 500to 600 pound with the improving australian dollar the air fare seem tobe more expensive aswell so I dont understand why they can be cheaper to purchase in the uk any ideas will be much appreciated some fares in this country approaching $2000 aud at the cheap end cheers",0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
28057,how can get my new credit card shipped to a different address?,2,"Credit card issues, internet problem, airport, airplane trip booking"
4494,Hi..I have a one pass miles that I never used and how do i go about seeing if I can use these miles for a free ticket to florida?,1,"Long haul flight, one way ticket, miles/points usage, finding train station, payment, password reset"
2858,especific picture of marylin monroe black and white rectangle forgotten in jan 7 2013 from houston to guadalajara mex,1,"Long haul flight, one way ticket, miles/points usage, finding train station, payment, password reset"
2786,I was charged twice for baggage on my flight who do I contact?,2,"Credit card issues, internet problem, airport, airplane trip booking"
16544,Have not received a bill so I do not know my username,0,"Round trip tickets, xml file appearing, redeeming miles/points, claim refusal, change of password"
6180,I would like to be able to access this account online: --------,3,"Sign-in issues, xml message, service issues, slow internet, phone number, cable box, DVR"
